In [15]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray
import xarray.ufuncs as xu
import dask
import seaborn as sn
from datetime import datetime
from dask.diagnostics import ProgressBar
import dask
import warnings
import os
import scipy
from pytz import timezone
import re

western = timezone('US/Pacific')
utc = timezone('UTC')

In [16]:
ds1 = xarray.open_dataset('/home/araldif/huckley/past_1980_Tsurface.nc')
ds1

<xarray.Dataset>
Dimensions:      (shade: 5, soil_mic: 19, soil_wrf: 4, south_north: 129, time: 8760, west_east: 199)
Coordinates:
  * time         (time) float64 1.98e+09 1.98e+09 1.98e+09 1.98e+09 1.98e+09 ...
  * shade        (shade) float64 0.0 25.0 50.0 75.0 100.0
  * soil_mic     (soil_mic) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18
  * soil_wrf     (soil_wrf) int64 0 1 2 3
  * south_north  (south_north) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...
  * west_east    (west_east) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
Data variables:
    lon          (south_north, west_east) float64 -131.4 -131.1 -130.8 ...
    lat          (south_north, west_east) float64 12.36 12.45 12.54 12.64 ...
    layers_wrf   (soil_wrf) float64 0.05 0.1 0.4 1.0
    layers_mic   (soil_mic) float64 0.03 0.06 0.09 0.12 0.15 0.18 0.21 0.24 ...
    Tsurface     (shade, time, south_north, west_east) float64 nan nan nan ...
Attributes:
    CREATION_DATE: 2015-12-18 03:59:01 AM CST

In [17]:
ds1.__delitem__('soil_mic')

In [18]:
ds1.__delitem__('soil_wrf')

In [19]:
ds1

<xarray.Dataset>
Dimensions:      (shade: 5, south_north: 129, time: 8760, west_east: 199)
Coordinates:
  * time         (time) float64 1.98e+09 1.98e+09 1.98e+09 1.98e+09 1.98e+09 ...
  * shade        (shade) float64 0.0 25.0 50.0 75.0 100.0
  * south_north  (south_north) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...
  * west_east    (west_east) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
Data variables:
    lon          (south_north, west_east) float64 -131.4 -131.1 -130.8 ...
    lat          (south_north, west_east) float64 12.36 12.45 12.54 12.64 ...
    Tsurface     (shade, time, south_north, west_east) float64 nan nan nan ...
Attributes:
    CREATION_DATE: 2015-12-18 03:59:01 AM CST

In [20]:
dsDates = ds1.time.to_series()

In [7]:
subset = ds1.sel(shade=0.0)

In [8]:
subset.Tsurface.values

array([[[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       [[ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        ..., 
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan],
        [ nan,  nan,  nan, ...,  nan,  nan,  nan]],

       ..., 
       [[ nan,  nan, 

In [21]:
ds1Tsurf = xarray.Dataset({'Tsurf': (['shade','time','x','y'], ds1.Tsurface.values)}, coords = {'longitude': (['x','y'], ds1.lon.values),
                                           'latitude': (['x','y'], ds1.lat.values),
                                           'time': xarray.DataArray.from_series(dsDates).values})

MemoryError: 

In [10]:
ds1Tsurf

<xarray.Dataset>
Dimensions:    (shade: 5, time: 8760, x: 129, y: 199)
Coordinates:
  * time       (time) float64 1.98e+09 1.98e+09 1.98e+09 1.98e+09 1.98e+09 ...
    longitude  (x, y) float64 -131.4 -131.1 -130.8 -130.5 -130.2 -129.9 ...
    latitude   (x, y) float64 12.36 12.45 12.54 12.64 12.73 12.82 12.91 13.0 ...
  * shade      (shade) int64 0 1 2 3 4
  * x          (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * y          (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
Data variables:
    Tsurf      (shade, time, x, y) float64 nan nan nan nan nan nan nan nan ...

In [11]:
ds2 = xarray.open_dataset('/home/araldif/huckley/past_1981_Tsurface.nc')
ds2.__delitem__('soil_mic')
ds2.__delitem__('soil_wrf')
dsDates2 = ds2.time.to_series()

In [12]:
ds2Tsurf = xarray.Dataset({'Tsurf': (['shade','time','x','y'], ds2.Tsurface.values)}, coords = {'longitude': (['x','y'], ds2.lon.values),
                                           'latitude': (['x','y'], ds2.lat.values),
                                           'time': xarray.DataArray.from_series(dsDates2).values})

In [13]:
ds3 = xarray.open_dataset('/home/araldif/huckley/past_1982_Tsurface.nc')
ds3.__delitem__('soil_mic')
ds3.__delitem__('soil_wrf')
dsDates3 = ds3.time.to_series()

In [14]:
ds3Tsurf = xarray.Dataset({'Tsurf': (['shade','time','x','y'], ds3.Tsurface.values)}, coords = {'longitude': (['x','y'], ds3.lon.values),
                                           'latitude': (['x','y'], ds3.lat.values),
                                           'time': xarray.DataArray.from_series(dsDates3).values})

MemoryError: 